### Below are the following assumptions

- A325 Bolts and A563 2H Nuts are used in all connections
- Hole sizes are in accordance to AISC J3.2


### Below is everything that needs checked

- holes must meet AISC J3.2 requirements XX
- holes vertical and horizontal spacing must meet AISC J3.4 XX
- bolt shear
- bolt bearing
- bolt tearout
- determine max plate thickness
- plate shear yielding
- plate shear rupture
- plate block shear rupture
- plate flexural rupture
- beam web shear yielding
- beam web shear rupture
- beam web block shear rupture
- beam web flexural rupture
- check plate for limit states defined by AISC equation 10-5
- check plate for buckling using the double-coped beam procedure from part 9

In [30]:
#function definition
def findBeamRow(df,beamName):
    beamRow = df[df["Name"] == beamName].to_dict(orient="records")[0]
    return beamRow

def findBoltRow(df,boltDia):
    boltRow = df[df["dia"] == boltDia].to_dict(orient="records")[0]
    return boltRow

def getBoltLength(plateThk,webThk,nutHeight):
    boltLength = plateThk + webThk + nutHeight
    boltLength = math.ceil(boltLength*4)/4
    return boltLength

def checkBoltEdgeDistance(minEdgeDistance,dist,dimensionName):
    print(f"{dimensionName} bolt edge distance check")
    if dist >= minEdgeDistance:
        print(f"\tPASS:{d_ct} >= {minEdgeDistance}\n")
    else:
        print(f"\tFAIL: {d_ct} < {minEdgeDistance}\n")
    return

def getBoltShearStrength(boltLen,plateThk,webThk,threadLen):
    shankLength = boltLen-threadLen
    shearPlane = min(plateThk,webThk) #assume the bolts are going in the "worst" direction

    #bolt shear strengths from table J3.2
    if shearPlane > shankLength: # threads are not in shear plane
        shearStrength = 68
    else:
        shearStrength = 54
    return shearStrength

def checkBoltShearStrength(boltArea,numBolts,shearStrength,load):
    R_ult = 0.75*shearStrength*boltArea #this is the nominal shear strength of a single bolt
    
    

In [26]:
shearLoad = 2  #value in kips
primaryBeam = "W8X31"
secondaryBeam = "W6X25"
beamGap = 0.5 #gap between top flange of the primary and secondary beam
plateThk = 0.375
boltDia = 0.625
numRow = 2
boltSpacing = 1.5

#bolt horizontal positions
L_ehb = 1.5  # center of bolt to end of secondary beam top flange (important if coped)
L_ehp = 1.5  # center of bolt to end of plate

#bolt vertical positions
L_evpt = 1.25 #distance from center of top hole to top edge of plate
L_evpb = 1.25 #distance from center of bottom hole to bottom edge of plate
d_ct = 1.75 #distance from center of top hole to top of beam

In [27]:
# import stuff...
import pandas as pd
import math

# import design tables
beamTable = pd.read_excel("Resources/aisc-shapes-database-v15.xlsx", engine="openpyxl", sheet_name=1, usecols="A,B,G,L,Q,T")
boltTable = pd.read_excel("Resources/structural-bolt-dimensions.xlsx",engine="openpyxl")

#connection variables

#get the row number for the primary and secondary beam
primaryBeamData = findBeamRow(beamTable,primaryBeam.upper())
secondaryBeamData = findBeamRow(beamTable,secondaryBeam.upper())

boltData = findBoltRow(boltTable,boltDia)
print(boltData["dia"])

boltLength = getBoltLength(plateThk,secondaryBeamData["tw"],boltData["nutHeight"])


0.625


In [28]:
#check dimensional constraints
#bolt edge distance from Table J3.4
minEdgeDistance = boltData["edgeDist"]
checkBoltEdgeDistance(minEdgeDistance,d_ct,"d_ct")
checkBoltEdgeDistance(minEdgeDistance,L_evpt,"L_evpt")
checkBoltEdgeDistance(minEdgeDistance,L_evpb,"L_evpb")
checkBoltEdgeDistance(minEdgeDistance,L_ehb,"L_ehb")
checkBoltEdgeDistance(minEdgeDistance,L_ehp,"L_ehp")


d_ct bolt edge distance check
	PASS:1.75 >= 0.875
L_evpt bolt edge distance check
	PASS:1.75 >= 0.875
L_evpb bolt edge distance check
	PASS:1.75 >= 0.875
L_ehb bolt edge distance check
	PASS:1.75 >= 0.875
L_ehp bolt edge distance check
	PASS:1.75 >= 0.875


In [ ]:
#check bolt strength
